In [1]:
import pandas as pd
import numpy as np
import joblib
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
import lightgbm as lgb
import warnings
from sklearn.preprocessing import KBinsDiscretizer
warnings.filterwarnings('ignore')

# from google.colab import drive
# drive.mount('/content/drive')

# ! pip install jquants-api-client shap
import jquantsapi
import shap


c:\Users\trow1\Documents\my_repositories\Nikkei_OpenClose_Quant\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dir_path = os.getcwd() + '/data/'

# 指定されたパスが存在しない場合にのみディレクトリを作成する
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

filepath_stock_price = os.path.join(dir_path, 'stock_price_load.gz')
filepath_stock_list   = os.path.join(dir_path, 'stock_list_load.gz')
filepath_stock_price_TOPIX500 = os.path.join(dir_path, 'stock_price_TOPIX500_load.gz')


In [3]:
# J-Quants API から取得するデータの期間。ライトプランを想定し，5年間のデータ取得としています。
# 無料プランの場合は2年間（２週間ラグあり）取得できます。
HISTORICAL_DATA_YEARS = 10

start_dt = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=(HISTORICAL_DATA_YEARS*365))
end_dt = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
start_dt_yyyymmdd = start_dt.strftime("%Y%m%d")
end_dt_yyyymmdd = end_dt.strftime("%Y%m%d")

print( f'{start_dt_yyyymmdd}から{end_dt_yyyymmdd}までの記録を使用します。' )

20150512から20250509までの記録を使用します。


In [4]:
import configparser
import os

# 設定ファイルを読み込む
config = configparser.ConfigParser()
config.read('token.ini')  # このファイルは.gitignoreに追加する


['token.ini']

In [5]:
# J-Quants APIクライアントの初期化
# cli = jquantsapi.Client()
cli = jquantsapi.Client(refresh_token=config["credentials"]["refresh_token"])

if os.path.exists(filepath_stock_list):
   # ファイルが存在する場合、読み込む
   stock_list_load = pd.read_csv(filepath_stock_list, compression='gzip')
else:
   # ファイルが存在しない場合、新たに作成する
   stock_list_load = cli.get_listed_info()
   stock_list_load.to_csv(filepath_stock_list, compression='gzip', index=False)


# 株価情報のobject型をdatetime64[ns]型に変換
stock_list_load["Date"] = pd.to_datetime(stock_list_load["Date"])

# 普通株 (5桁で末尾が0) の銘柄コードを4桁にします
stock_list_load["Code"] = stock_list_load["Code"].astype(str)
stock_list_load.loc[(stock_list_load["Code"].str.len() == 5) & (stock_list_load["Code"].str[-1] == "0"), "Code"] = stock_list_load.loc[(stock_list_load["Code"].str.len() == 5) & (stock_list_load["Code"].str[-1] == "0"), "Code"].str[:-1]
# stock_list_load["Code"] = stock_list_load["Code"].astype(int)

# TOPIX500のtickerのみを取得する
categories = ['TOPIX Mid400', 'TOPIX Large70', 'TOPIX Core30']
tickers = stock_list_load[stock_list_load['ScaleCategory'].isin(categories)]['Code'].unique()
tickers = tickers.astype(str)

In [6]:
if os.path.exists(filepath_stock_price):
   # ファイルが存在する場合、読み込む
   stock_price_load = pd.read_csv(filepath_stock_price, compression='gzip')
else:
   # ファイルが存在しない場合、新たに作成する。初回は10分以上かかると思います。
   stock_price_load = cli.get_price_range(start_dt=start_dt, end_dt=end_dt)
   stock_price_load.to_csv(filepath_stock_price, compression='gzip', index=False)

# "Code"列を文字列型に変換
stock_price_load["Code"] = stock_price_load["Code"].astype(str)
# 普通株 (5桁で末尾が0) の銘柄コードを4桁にします
stock_price_load.loc[(stock_price_load["Code"].str.len() == 5) & (stock_price_load["Code"].str[-1] == "0"), "Code"] = stock_price_load.loc[(stock_price_load["Code"].str.len() == 5) & (stock_price_load["Code"].str[-1] == "0"), "Code"].str[:-1]

display(stock_price_load)

,Date,Code,Open,High,Low,Close,UpperLimit,LowerLimit,Volume,TurnoverValue,AdjustmentFactor,AdjustmentOpen,AdjustmentHigh,AdjustmentLow,AdjustmentClose,AdjustmentVolume
0,2015-05-11,1301,281.0,291.0,281.0,290.0,0,0,955000.0,275195000.0,1.0,2810.0,2910.0,2810.0,2900.0,95500.0
1,2015-05-12,1301,285.0,291.0,284.0,288.0,0,0,451000.0,129905000.0,1.0,2850.0,2910.0,2840.0,2880.0,45100.0
2,2015-05-13,1301,286.0,289.0,281.0,283.0,0,0,351000.0,100089000.0,1.0,2860.0,2890.0,2810.0,2830.0,35100.0
3,2015-05-14,1301,282.0,286.0,281.0,282.0,0,0,281000.0,79516000.0,1.0,2820.0,2860.0,2810.0,2820.0,28100.0
4,2015-05-15,1301,283.0,285.0,282.0,283.0,0,0,90000.0,25480000.0,1.0,2830.0,2850.0,2820.0,2830.0,9000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9915737,2025-04-30,9997,975.0,984.0,967.0,980.0,0,0,264600.0,258591800.0,1.0,975.0,984.0,967.0,980.0,264600.0
9915738,2025-05-01,9997,972.0,972.0,956.0,958.0,0,0,311100.0,299767700.0,1.0,972.0,972.0,956.0,958.0,311100.0
9915739,2025-05-02,9997,956.0,962.0,926.0,934.0,0,0,253800.0,238089000.0,1.0,956.0,962.0,926.0,934.0,253800.0
9915740,2025-05-07,9997,935.0,939.0,930.0,934.0,0,0,163400.0,152761700.0,1.0,935.0,939.0,930.0,934.0,163400.0


In [7]:
df_ohlcv = stock_price_load[stock_price_load['Code'].isin(tickers)]

if os.path.exists(filepath_stock_price_TOPIX500):
   # ファイルが存在する場合、読み込む
   stock_list_load = pd.read_csv(filepath_stock_price_TOPIX500, compression='gzip')
else:
   # ファイルが存在しない場合、新たに作成する
   stock_list_load = cli.get_listed_info()
   stock_list_load.to_csv(filepath_stock_price_TOPIX500, compression='gzip', index=False)


display(df_ohlcv)

,Date,Code,Open,High,Low,Close,UpperLimit,LowerLimit,Volume,TurnoverValue,AdjustmentFactor,AdjustmentOpen,AdjustmentHigh,AdjustmentLow,AdjustmentClose,AdjustmentVolume
49028,2015-05-11,1332,370.0,373.0,366.0,369.0,0,0,1987300.0,7.336529e+08,1.0,370.0,373.0,366.0,369.0,1987300.0
49029,2015-05-12,1332,367.0,373.0,366.0,372.0,0,0,1730800.0,6.411681e+08,1.0,367.0,373.0,366.0,372.0,1730800.0
49030,2015-05-13,1332,372.0,376.0,368.0,375.0,0,0,1790300.0,6.685820e+08,1.0,372.0,376.0,368.0,375.0,1790300.0
49031,2015-05-14,1332,375.0,379.0,373.0,376.0,0,0,2688200.0,1.012012e+09,1.0,375.0,379.0,373.0,376.0,2688200.0
49032,2015-05-15,1332,380.0,393.0,371.0,376.0,0,0,6417100.0,2.440305e+09,1.0,380.0,393.0,371.0,376.0,6417100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9897810,2025-04-30,9989,4711.0,4800.0,4711.0,4784.0,0,0,528900.0,2.523012e+09,1.0,4711.0,4800.0,4711.0,4784.0,528900.0
9897811,2025-05-01,9989,4737.0,4738.0,4694.0,4734.0,0,0,312500.0,1.475452e+09,1.0,4737.0,4738.0,4694.0,4734.0,312500.0
9897812,2025-05-02,9989,4673.0,4758.0,4654.0,4696.0,0,0,343200.0,1.614028e+09,1.0,4673.0,4758.0,4654.0,4696.0,343200.0
9897813,2025-05-07,9989,4719.0,4799.0,4699.0,4745.0,0,0,559700.0,2.662514e+09,1.0,4719.0,4799.0,4699.0,4745.0,559700.0
